In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../")

In [115]:
import glob
from tqdm.notebook import trange, tqdm
import re
import os.path
import numpy as np
from features import prepare_entry
from features import merge_entries
from sklearn.preprocessing import StandardScaler
from features import merge_entries, get_transformer
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

In [3]:
DATA_PATH = '../data/wikipedia/'

In [63]:
def wiki_preprocess(text):
    text = re.sub(r'\[\[User:.*\]\]\s\(\[\[User.*\]\].*\(UTC\)', '', text)
    text = re.sub(r'\[\[.*\]\]', 'PAGE', text)
    text = re.sub(r'\d{2}:\d{2},\s\d+\s\w+\s\d{4}\s\(UTC\)', 'DATE', text)
    text = re.sub(r'==.*==(\n)*', '', text)
    
    return text

In [99]:
doc_pairs = []
username_pairs = []
labels = []
for directory in glob.glob(DATA_PATH + 'Sockpuppet/*'):
    fps = glob.glob(os.path.join(directory, '*'))
    usernames = [os.path.basename(os.path.normpath(fp)) for fp in fps]
    files_1 = [f for f in glob.glob(os.path.join(fps[0], '*.*')) if 'Categories.txt' not in f and 'Articles.txt' not in f]
    files_2 = [f for f in glob.glob(os.path.join(fps[1], '*.*')) if 'Categories.txt' not in f and 'Articles.txt' not in f]
    docs_1 = [wiki_preprocess(open(f).read()) for f in files_1]
    docs_2 = [wiki_preprocess(open(f).read()) for f in files_2]
    if len('\n'.join(docs_1)) < 10 or  len('\n'.join(docs_2)) < 10:
        continue
    doc_pairs.append((docs_1, docs_2))
    username_pairs.append(usernames)
    labels.append(1)
    
for directory in glob.glob(DATA_PATH + 'Non-sockpuppet/*'):
    fps = glob.glob(os.path.join(directory, '*'))
    usernames = [os.path.basename(os.path.normpath(fp)) for fp in fps]
    files_1 = [f for f in glob.glob(os.path.join(fps[0], '*.*')) if 'Categories.txt' not in f and 'Articles.txt' not in f]
    files_2 = [f for f in glob.glob(os.path.join(fps[1], '*.*')) if 'Categories.txt' not in f and 'Articles.txt' not in f]
    docs_1 = [wiki_preprocess(open(f).read()) for f in files_1]
    docs_2 = [wiki_preprocess(open(f).read()) for f in files_2]
    if len('\n'.join(docs_1)) < 10 or  len('\n'.join(docs_2)) < 10:
        continue
    doc_pairs.append((docs_1, docs_2))
    username_pairs.append(usernames)
    labels.append(0)

In [100]:
docs_1_preprocessed = [prepare_entry('\n'.join(d1)) for d1, d2 in tqdm(doc_pairs)]
docs_2_preprocessed = [prepare_entry('\n'.join(d2)) for d1, d2 in tqdm(doc_pairs)]

In [134]:
docs_1_preprocessed = np.array(docs_1_preprocessed)
docs_2_preprocessed = np.array(docs_2_preprocessed)
username_pairs = np.array(username_pairs)

In [135]:
p = np.random.permutation(len(labels))
labels = labels[p]
docs_1_preprocessed = docs_1_preprocessed[p]
docs_2_preprocessed = docs_2_preprocessed[p]
username_pairs = username_pairs[p]

In [142]:
kf.get_n_splits()

10

In [143]:
aucs = []
kf = KFold(n_splits=20)
for train_index, test_index in tqdm(kf.split(labels), total=kf.get_n_splits()):
    transformer = get_transformer()
    scaler = StandardScaler()
    X = transformer.fit_transform(np.concatenate([docs_1_preprocessed[train_index], docs_2_preprocessed[train_index]]))
    X = scaler.fit_transform(X.todense())
    X1 = X[:len(train_index)]
    X2 = X[len(train_index):]

    secondary_scaler = StandardScaler()
    X_diff = np.abs(X1 - X2)
    X_diff = secondary_scaler.fit_transform(X_diff)
    
    clf = SGDClassifier(loss='log', alpha=0.01)
    clf.fit(X_diff, labels[train_index])
    
    X1 = scaler.transform(transformer.transform(docs_1_preprocessed[test_index]).todense())
    X2 = scaler.transform(transformer.transform(docs_2_preprocessed[test_index]).todense())
    X_diff_test = secondary_scaler.transform(np.abs(X1 - X2))
    probs = clf.predict_proba(X_diff_test)[:, 1]
    fpr, tpr, thresh = roc_curve(labels[test_index], probs)
    roc_auc = auc(fpr, tpr)
    print('AUC:', roc_auc)
    aucs.append(roc_auc)

AUC: 0.7583333333333333
AUC: 0.79296875
AUC: 0.7411764705882353
AUC: 0.8458333333333332
AUC: 0.7686274509803921
AUC: 0.9138655462184874
AUC: 0.6666666666666666
AUC: 0.8179824561403509
AUC: 0.8376068376068376
AUC: 0.8844537815126051
AUC: 0.7815126050420168
AUC: 0.7166666666666668
AUC: 0.8636363636363636
AUC: 0.8290598290598291
AUC: 0.7456140350877193
AUC: 0.9411764705882353
AUC: 0.7815126050420169
AUC: 0.8289473684210527
AUC: 0.8109243697478992
AUC: 0.8



In [111]:
labels = np.array(labels)

In [113]:
X_train, X_test, y_train, y_test = train_test_split(X_diff, labels, test_size=0.33, random_state=42)
clf = SGDClassifier(loss='log', alpha=0.01)
clf.fit(X_train, y_train)

probs = clf.predict_proba(X_test)[:, 1]
fpr, tpr, thresh = roc_curve(y_test, probs)
roc_auc = auc(fpr, tpr)
print('AUC:', roc_auc)

AUC: 0.8481557760718966


In [140]:
len(fnames)

21126

In [110]:
fnames = np.array(transformer.get_feature_names())
fnames[np.argsort(-np.abs(clf.coef_[0]))][:50]

array(['char_distr__0p', 'char_distr__gr', "pos_tag_distr__'' VBD ''",
       "masked_stop_words_distr__'' VBD ''", 'freq_func_words__across',
       'masked_stop_words_distr__in such',
       'masked_stop_words_distr__much -None-',
       'masked_stop_words_distr__it might',
       'masked_stop_words_distr__JJ page',
       'masked_stop_words_distr__: for', 'pos_tag_distr__NN IN VB',
       'masked_stop_words_distr__NNS of ``',
       'masked_stop_words_distr__not really',
       'masked_stop_words_distr__-None- date :',
       'masked_stop_words_distr__across', 'char_distr__ :f',
       'masked_stop_words_distr__. also',
       'masked_stop_words_distr__for -None-',
       'pos_tag_chunks_subtree_distr__VP[VBN]',
       'masked_stop_words_distr__: is', 'char_distr__-20',
       'pos_tag_distr__JJR CC JJR', 'char_distr__so',
       'masked_stop_words_distr__NNS and the', 'char_distr__-ne',
       'pos_tag_distr__TO PRP IN', 'char_distr__i> ',
       'masked_stop_words_distr__: i would